In [1]:
from env import Workload
from env.workflow import Workflow
from schedule.taskNcpEnv import runEnv
from schedule.schedule import Scheduler
from model.sac import SAC
from model.dqn import DQN
import torch
import torch.nn as nn
import datetime
import time  


# __init__ 初始化
random_seed = 50
buffer_size = 1000
batch_size = 512
hidden_size = 128
target_update = 100
action_num = 16
state_dim =  6+3*action_num 
# 将某个事件的到达率从每分钟 0.1 次转换为每秒的到达率
arrival_rate = 0.1/60
train_path = 'data/train'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
episode_number = 100
wf_number = 10
# 平衡预算和时间的参数
beta = 0.6
# 平衡未来奖励的权重
discount_factor = 0.8
lr = 1e-3
# L2正则化的权重
l2_weight_decay = 1e-4
debug = True

19344587


In [2]:
train_dataset_50 = "data/train/_50"
agent1 = DQN(state_dim, action_num, batch_size, buffer_size, hidden_size, lr, device, discount_factor);
agent2 = SAC(state_dim, action_num, batch_size, buffer_size, hidden_size, lr, device)
agent3 = DQN(state_dim, action_num, batch_size, buffer_size, hidden_size, lr, device, discount_factor, l2_weight_decay); #R-DQN
save_model_path1 = 'result/dqn_model.pth'
save_model_path2 = 'result/sac_model.pth'
save_model_path_d50 = 'result/dqn_model_50.pth'
save_model_path_s50 = 'result/sac_model_50.pth'

In [3]:
def train(episode_number, agent, workflow_number, train_wf_path, save_model_path,arrival_rate, debug=False):
  mean_makespan = [];
  mean_cost = []
  time_rate = []
  cost_rate = []
  succes_both_rate = []
  episode_arr = []
  
  agent.net.train(True)
  scheduler = Scheduler(agent, action_num, state_dim, batch_size, beta)
  print("start at:", str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")));
  start = time.time();
        
  for episode in range(1, episode_number+1):
    Workflow.reset();
    Workload.reset();
  
    print("episode:",episode,"="*70)
    t, c, tr, cr, both = runEnv(train_wf_path, scheduler.run, episode*10, wf_number=workflow_number, action_num = action_num, 
                                  arrival_rate = arrival_rate, debug=debug);
    mean_makespan.append(t)
    mean_cost.append(c)
    time_rate.append(tr)
    cost_rate.append(cr)
    succes_both_rate.append(both)
    episode_arr += ([episode])
    
        
  s = str(datetime.timedelta(seconds=time.time()-start));
  print("total train time:", s);
  agent.save_model(save_model_path)
  scheduler.trainPlotFinal(mean_makespan, mean_cost,time_rate, cost_rate, succes_both_rate);

In [4]:
train(episode_number, agent1, wf_number, train_dataset_50, save_model_path1, arrival_rate, debug);

start at: 2024-09-26 20:49:50
episode: 1 ======================================================================
[0.00 - vm0.1     ] Start.
[0.00 - vm1.1     ] Start.
[0.00 - vm2.1     ] Start.
[0.00 - vm3.1     ] Start.
[0.00 - vm4.1     ] Start.
[0.00 - vm5.1     ] Start.
[0.00 - vm6.1     ] Start.
[0.00 - vm7.1     ] Start.
[0.00 - vm8.1     ] Start.
[0.00 - vm9.1     ] Start.
[0.00 - vm10.1    ] Start.
[0.00 - vm11.1    ] Start.
[0.00 - vm12.1    ] Start.
[0.00 - vm13.1    ] Start.
[0.00 - vm14.1    ] Start.
[0.00 - vm15.1    ] Start.
[618.00 - Pool      ] data/train/_50/MONTAGE.n.50.0.dax (id: wf1, deadline: 3708.10, budget: 8.68, df: 13.00, bf: 3.00) is saved in the pool.
 # current Wf:1 # total Wf:1
[618.00 - ReadyQueue] 8 tasks are added to ready queue. queue size: 8.
[task deadline] task_len :70.16914402870323, remained_deadline :3708.1000000000004, deadline :913.3481288233178
[task deadline] task_len :70.37416709379805, remained_deadline :3708.1000000000004, deadline :916.1816

KeyboardInterrupt: 